In [1]:
%reload_ext autoreload
%autoreload 2
%cd ..

c:\Users\heroe\OneDrive\桌面\play-gpt\vnov


C:\Users\heroe\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import PIL.Image as pil
import moviepy.editor as mp
from moviepy.video.tools.subtitles import SubtitlesClip, TextClip
import os
import re
from moviepy.config import change_settings
pil.ANTIALIAS = pil.Resampling.LANCZOS
change_settings({"IMAGEMAGICK_BINARY": r"C:\\Program Files\\ImageMagick-7.1.1-Q16-HDRI\\magick.exe"})

In [3]:
def parse_srt(srt_file, encoding="utf-8"):
    """Parses the SRT file into a list of (start_time, end_time, subtitle_text) tuples."""
    pattern = re.compile(r"(\d{2}):(\d{2}):(\d{2}),(\d{3})")
    subtitles = []

    with open(srt_file, 'r', encoding=encoding) as f:
        lines = f.readlines()

    current_sub = {}
    for line in lines:
        # Match time codes
        if pattern.search(line):
            times = pattern.findall(line)
            start_time = int(times[0][0]) * 3600 + int(times[0][1]) * 60 + int(times[0][2]) + int(times[0][3]) / 1000
            end_time = int(times[1][0]) * 3600 + int(times[1][1]) * 60 + int(times[1][2]) + int(times[1][3]) / 1000
            current_sub['start'] = start_time
            current_sub['end'] = end_time
        elif line.strip() == '':
            if 'text' in current_sub:
                # Remove prefix numbers and keep only the subtitle text
                text = re.sub(r'^\d+\s*', '', current_sub['text'].strip())
                subtitles.append((current_sub['start'], current_sub['end'], text))
            current_sub = {}
        else:
            if 'text' not in current_sub:
                current_sub['text'] = line.strip()
            else:
                current_sub['text'] += ' ' + line.strip()
    
    return subtitles


In [31]:
def create_video_with_audio_and_subtitles(image_file, audio_file, subtitle_file, output_file):
    # Load the audio
    audio = mp.AudioFileClip(audio_file)
    
    # Load the image
    image_clip = mp.ImageClip(image_file).set_duration(audio.duration)  # Set the duration to match audio

    # Resize the image to maintain a 16:9 aspect ratio (1920x1080) while preserving its aspect ratio
    target_width = 12/9*1080 // 1
    target_height = 1080
    image_clip = image_clip.resize(height=target_width)  # Set height to 1920x1920

    # Create a video clip with zooming and vertical panning effect
    def make_frame(t):
        y_offset = (image_clip.h - target_height) * (t / audio.duration)  # Calculate vertical pan offset
        frame = image_clip.set_position(('center', 'top')).get_frame(t)
        return frame[int(y_offset):int(y_offset + target_height), :, :]
    

    # Create a video clip with the zoom and vertical panning effect
    video_clip = mp.VideoClip(make_frame, duration=audio.duration)

    # Set the audio for the video clip
    video_clip = video_clip.set_audio(audio)

    # Parse the subtitles manually
    subtitles = parse_srt(subtitle_file)

    # Create TextClip objects for each subtitle and overlay them
    subtitle_clips = []
    for start, end, text in subtitles:
        # Use the specified custom font
        txt_clip = TextClip(text, font='./datasets/simyou.ttf', fontsize=40, color='white')
        txt_clip = txt_clip.set_position(('center', 'bottom')).set_start(start).set_duration(end - start)
        subtitle_clips.append(txt_clip)

    # Composite the video and subtitles
    final_video = mp.CompositeVideoClip([video_clip] + subtitle_clips)
    final_video = final_video.subclip(0, max(0, final_video.duration - 0.05))
    
    # Export the final video
    final_video.write_videofile(output_file, fps=24, codec='libx264', audio_codec='aac', audio_bitrate='192k', preset='ultrafast')



In [32]:
# Example usage:
image_file = "./datasets/yujie/scene_images/scene3.png"  # Replace with the path to your PNG image
audio_file = "./datasets/yujie/yujie_n/yujie_tts_3/yujie_tts_3.mp3"  # Replace with the path to your MP3 audio file
subtitle_file = "./datasets/yujie/yujie_n/yujie_tts_3/yujie_tts_3.srt"  # Replace with the path to your SRT subtitle file
output_file = "./temp.mp4"  # Replace with the desired output video file name

create_video_with_audio_and_subtitles(image_file, audio_file, subtitle_file, output_file)

t:   1%|          | 3/516 [07:40<21:53:29, 153.63s/it, now=None]

Moviepy - Building video ./temp.mp4.
MoviePy - Writing audio in tempTEMP_MPY_wvf_snd.mp4


t:   1%|          | 3/516 [07:41<21:55:23, 153.85s/it, now=None]

MoviePy - Done.
Moviepy - Writing video ./temp.mp4



t:   1%|          | 3/516 [07:52<22:26:14, 157.46s/it, now=None]

Moviepy - Done !
Moviepy - video ready ./temp.mp4
